# **Open Source SW and Python Programming Project: Implementation of Subway Navigation**

In this project, you will implement a navigation system for Seoul subway stations, especially **step by step**




# 0. Initialization

Read subway station information by reading the file **simplified_subway_info_english.xlsx** or **simplified_subway_info_korean.xlsx**
*  The excel file contains subway station information of Seoul subway line 1 ~ 4
*  When you execute this sourcecode, you MUST upload these files in your Colab runtime environment
*   **MUST NOT** change this code cell

In [271]:
import xlrd

# Read data file
data = xlrd.open_workbook("simplified_subway_info_korean.xlsx")
data = data.sheet_by_name('Sheet1')

# Store the loaded book object as a string list in subwayStation variable
subwayStation = []
for line in range(4) :
    cur = [x for x in data.col_values(line) if x]    
    subwayStation.append(cur[1:])

# 1. Your implementation
Let's start the implementation of a subway naviation system by using the loaded subway station information :)

*   You can access the subway station information by referring to **subwayStation** variable (list type)

In [ ]:
subwayStation = {'Line1':tuple((subwayStation[0])),'Line2':tuple((subwayStation[1])),'Line3':tuple((subwayStation[2])),'Line4':tuple((subwayStation[3]))}
for key in subwayStation.keys() :
    print(key)
for value in subwayStation.values() :
    print(value)

In [273]:
def Login() :
    print("\n***********************************************")
    print("Log in (or sign up)")
    print("***********************************************")

    username = input(">> Please enter username : ")
    password = input(">> Please enter password : ")

    with open('users_info.txt', 'r') as f:
        for line in f:
            user_info = line.split(',')
            if username == user_info[0]:
                while True:
                    if password == user_info[1]:   #로그인 성공
                        return username
                    else:                       #비밀번호 틀림        
                        print("Password is incorrect.")
                        password = input(">> Please enter password again : ")
        
        #새로운 아이디인 경우
        with open('users_info.txt', 'a') as f:
            f.write(username + ',' + password + ', , ,\n')
            return username


def dispSubwayLineInfo() :
    option1_lineNum = None
    print("\n***********************************************")
    print("Subway line information service")
    print("***********************************************")
     
    while option1_lineNum not in range(1,5):
        try:
            option1_lineNum = int(input(">> Please enter a subway line number (1 - 4): "))
        except Exception as e:
            print('Error :', e)
            print("Please enter a vaild number")

    print(subwayStation['Line'+ str(option1_lineNum)])
    print("\n")


def stationIsValid(stationName) :
    isValid = False
    for i in range(1,5):
            if stationName in subwayStation['Line' + str(i)]:
                isValid = True 
    return isValid


def dispSubwayStationInfo() :
    option2_stationName = None
    isValid = False
    count = 0
    print("\n***********************************************")
    print("Subway station information service")
    print("***********************************************")
   
    while not isValid :
        option2_stationName = input(">> Please enter a subway station name: ")
        isValid = stationIsValid(option2_stationName)
            
    print(option2_stationName, "station is in Line" , end = " ")
   
    for i in range(1,5):
         if option2_stationName in subwayStation['Line' + str(i)]:
            count += 1
            if count != 1:
                print("and", end = " ")
            print(i, end = " ")
    print("\n")


def findPath(username) :
    departureIsValid = False
    arrivalIsValid = False
    print("\n***********************************************")
    print("Finding path service")
    print("***********************************************")

    while not departureIsValid :
        option3_departure = input(">> Please enter a departure subway station: ")
        departureIsValid = stationIsValid(option3_departure) 
    
    while not arrivalIsValid :
        option3_arrival = input(">> Please enter a arrival subway station: ")
        arrivalIsValid = stationIsValid(option3_arrival) 

    findPathFunc(option3_departure, option3_arrival)

    #파일에 최근 검색 기록 저장하기
    with open('users_info.txt', 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            user_info = lines[i].split(',')
            if username == user_info[0]:
                user_info[2] = option3_departure
                user_info[3] = option3_arrival
                lines[i] = ",".join(user_info)  #해당 라인만 수정
                break
    with open('users_info.txt', 'w') as f:
        f.writelines(lines)


def findPathFunc(departure, arrival):
    departureInLine = []
    arrivalInLine = []
    sameLine = 0
    departureIndex = 0
    arrivalIndex = 0
    lineTuple = ()

    #어느 라인에 들어있는지 확인
    for i in range(1,5):
        if departure in subwayStation['Line' + str(i)]:
            departureInLine.append(i)
        if arrival in subwayStation['Line' + str(i)]:
            arrivalInLine.append(i)

    
    #같은 라인이 있는지 탐색
    for d in range(len(departureInLine)):
        for a in range(len(arrivalInLine)):
            if departureInLine[d] == arrivalInLine[a]: #같은 라인이 있으면 
                sameLine = departureInLine[d]
                lineTuple = subwayStation['Line' + str(sameLine)]
                departureIndex = lineTuple.index(departure)
                arrivalIndex = lineTuple.index(arrival)

    def printPath(line, departureIndex, arrivalIndex):
        lineTuple = subwayStation['Line' + str(line)]
        if departureIndex <= arrivalIndex:  #순행이면
            print(lineTuple[departureIndex], end=' -> ')
            print(lineTuple[departureIndex + 1 : arrivalIndex], end=' -> ')
            print(lineTuple[arrivalIndex])
        else:
            print(lineTuple[departureIndex], end='->')
            print(lineTuple[departureIndex - 1 : arrivalIndex : -1], end='->')
            print(lineTuple[arrivalIndex])  
        
    
    def findTransferStation(deLine, arLine):
        lineTuple = subwayStation['Line' + str(deLine)]
        
        for i in range(len(lineTuple)):
            if lineTuple[i] in subwayStation['Line' + str(arLine)]:
                return lineTuple[i]


    #둘이 같은 라인에 있다면
    if sameLine != 0:
        printPath(sameLine, departureIndex, arrivalIndex)
       
    #둘이 다른 라인에 있다면 
    else:
        lineTuple = subwayStation['Line' + str(departureInLine[0])]
        departureIndex = lineTuple.index(departure)
        printPath(departureInLine[0], departureIndex, lineTuple.index(findTransferStation(departureInLine[0], arrivalInLine[0])))

        lineTuple = subwayStation['Line' + str(arrivalInLine[0])]
        arrivalIndex = lineTuple.index(arrival)
        print('[',arrivalInLine[0], '호선 환승',']' , end = ' ')
        printPath(arrivalInLine[0],lineTuple.index(findTransferStation(departureInLine[0], arrivalInLine[0])), arrivalIndex)


def viewRequest(username) :
    viewOption = None
    print("\n***********************************************")
    print("My recent requests service")
    print("***********************************************")

    with open('users_info.txt', 'r') as f:
        for line in f:
            user_info = line.split(',')
            if user_info[0] == username:
                departure = user_info[2]
                arrival = user_info[3]

    if departure == " ":
        print("There is no previous navigation request")
    else:
        print("Your most recent navigation request:")
        print("1. From {0} to {1}".format(departure, arrival))
        print("2. Go back")

        while viewOption not in range(1,3):
            try:
                viewOption = int(input("Please choose one of the options (1 - 2): "))
            except Exception as e:
                print('Error :', e)
                print("Please enter a vaild number")
            
        if viewOption == 1:
            findPathFunc(departure, arrival)


In [ ]:
loginOption = None
mainOption = None

print("****************************************************")
print("1. Log in (or sign up)")
print("2. Exit")
print("****************************************************")

while loginOption not in range(1,3):
    try:
        loginOption = int(input("Please choose one of the options (1 - 2): "))
    except Exception as e:
        print('Error :', e)
        print("Please enter a vaild number")

if loginOption == 1:
    username = Login()


while mainOption != 5 and loginOption != 2 :
    print("\n****************************************************")
    print("Welcome " + username + "!")
    print("1. Display subway line information (Line 1 ~ 4)")
    print("2. Display subway station information")
    print("3. Find a path between two subway stations")
    print("4. View my recent requests")
    print("5. Exit")
    print("****************************************************")

    while mainOption not in range(1,6):
        try:
            mainOption = int(input("Please choose one of the options (1 - 5): "))
        except Exception as e:
            print('Error :', e)
            print("Please enter a vaild number")

    if mainOption == 1:
        mainOption = None
        dispSubwayLineInfo()

    elif mainOption == 2:
        mainOption = None
        dispSubwayStationInfo()

    elif mainOption == 3:
        mainOption = None
        findPath(username)

    elif mainOption == 4:
        mainOption = None
        viewRequest(username)

        